## The pure BERT model for token classification

In [88]:
! python -V

Python 3.8.16


In [89]:
! pip install seqeval==0.0.12 #BERT-base-cased

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [90]:
! pip install pytorch-transformers

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [91]:
!pip install keras_preprocessing

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [92]:
! pip install scikit-learn

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [93]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForTokenClassification, AdamW

In [94]:
df_train = pd.read_csv("/kaggle/input/jargon-detection/train.csv",sep=",",encoding="latin1")
df_train.head()

,token_id,token,label
0,cs/doc_1/0/0,A,O
1,cs/doc_1/0/1,SHAP,O
2,cs/doc_1/0/2,summary,O
3,cs/doc_1/0/3,plot,TERM
4,cs/doc_1/0/4,shows,O


In [95]:
df_val = pd.read_csv("/kaggle/input/jargon-detection/validation.csv",sep=",",encoding="latin1")
df_val.head()

,token_id,token,label
0,cs/doc_1867/0/0,Section,O
1,cs/doc_1867/0/1,~,O
2,cs/doc_1867/0/2,illustrates,O
3,cs/doc_1867/0/3,the,O
4,cs/doc_1867/0/4,proposed,O


In [96]:
df_test = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")
df_test.shape

(42358, 3)

In [97]:
df_val.label.unique()

array(['O', 'TERM'], dtype=object)

In [98]:
df_train[['m','n','x','y']] = df_train["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [99]:
df_train.label.value_counts()

label
O       520286
TERM     54624
Name: count, dtype: int64

In [100]:
df_val.label.value_counts()

label
O       33479
TERM     3664
Name: count, dtype: int64

In [101]:
df_test.label.value_counts()

label
O    42358
Name: count, dtype: int64

In [102]:
df_val[['m','n','x','y']] = df_val["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [103]:
df_test[['m','n','x','y']] = df_test["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [104]:
df_train = df_train.fillna(' ') 

In [105]:
df_val = df_val.fillna(' ') 

In [106]:
df_test = df_test.fillna(' ') 

In [107]:
len(df_test)

42358

In [108]:
df_train[['a','b']] = df_train["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_train

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1/0/0,A,O,cs,doc_1,0,0,doc,1
1,cs/doc_1/0/1,SHAP,O,cs,doc_1,0,1,doc,1
2,cs/doc_1/0/2,summary,O,cs,doc_1,0,2,doc,1
3,cs/doc_1/0/3,plot,TERM,cs,doc_1,0,3,doc,1
4,cs/doc_1/0/4,shows,O,cs,doc_1,0,4,doc,1
...,...,...,...,...,...,...,...,...,...
574905,phys/doc_1866/5/20,from,O,phys,doc_1866,5,20,doc,1866
574906,phys/doc_1866/5/21,a,O,phys,doc_1866,5,21,doc,1866
574907,phys/doc_1866/5/22,commercial,O,phys,doc_1866,5,22,doc,1866
574908,phys/doc_1866/5/23,supercapacitor,TERM,phys,doc_1866,5,23,doc,1866


In [109]:
df_val[['a','b']] = df_val["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_val

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1867/0/0,Section,O,cs,doc_1867,0,0,doc,1867
1,cs/doc_1867/0/1,~,O,cs,doc_1867,0,1,doc,1867
2,cs/doc_1867/0/2,illustrates,O,cs,doc_1867,0,2,doc,1867
3,cs/doc_1867/0/3,the,O,cs,doc_1867,0,3,doc,1867
4,cs/doc_1867/0/4,proposed,O,cs,doc_1867,0,4,doc,1867
...,...,...,...,...,...,...,...,...,...
37138,phys/doc_1976/10/28,vorticity,TERM,phys,doc_1976,10,28,doc,1976
37139,phys/doc_1976/10/29,in,O,phys,doc_1976,10,29,doc,1976
37140,phys/doc_1976/10/30,the,O,phys,doc_1976,10,30,doc,1976
37141,phys/doc_1976/10/31,wake,O,phys,doc_1976,10,31,doc,1976


In [110]:
df_test[['a','b']] = df_test["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_test

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1977/0/0,Next,O,cs,doc_1977,0,0,doc,1977
1,cs/doc_1977/0/1,",",O,cs,doc_1977,0,1,doc,1977
2,cs/doc_1977/0/2,we,O,cs,doc_1977,0,2,doc,1977
3,cs/doc_1977/0/3,analyze,O,cs,doc_1977,0,3,doc,1977
4,cs/doc_1977/0/4,the,O,cs,doc_1977,0,4,doc,1977
...,...,...,...,...,...,...,...,...,...
42353,phys/doc_2100/10/20,also,O,phys,doc_2100,10,20,doc,2100
42354,phys/doc_2100/10/21,presented,O,phys,doc_2100,10,21,doc,2100
42355,phys/doc_2100/10/22,and,O,phys,doc_2100,10,22,doc,2100
42356,phys/doc_2100/10/23,discussed,O,phys,doc_2100,10,23,doc,2100


In [111]:
df_train = df_train.drop('y', axis=1)
df_train = df_train.drop('token_id', axis=1)
df_train = df_train.drop('n', axis=1)
df_train = df_train.drop('a', axis=1)

In [112]:
df_val = df_val.drop('y', axis=1)
df_val = df_val.drop('token_id', axis=1)
df_val = df_val.drop('n', axis=1)
df_val = df_val.drop('a', axis=1)

In [113]:
df_test = df_test.drop('y', axis=1)
df_test = df_test.drop('token_id', axis=1)
df_test = df_test.drop('n', axis=1)
df_test = df_test.drop('a', axis=1)

In [114]:
df_train["s"] =  df_train["b"] + df_train['x'].astype(str) 

In [115]:
df_val["s"] = df_train["b"] + df_val['x'].astype(str)

In [116]:
df_test["s"] = df_test["b"] + df_test['x'].astype(str) 

In [117]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 

In [118]:
df_train['c']= label_encoder.fit_transform(df_train['m'])

In [119]:
#Encode labels in column 'species'.
df_val['c']= label_encoder.fit_transform(df_val['m'])
df_val['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [120]:
# Encode labels in column 'species'.
df_test['c']= label_encoder.fit_transform(df_test['m'])
df_test['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [121]:
df_test["s"] = df_test['c'].astype(str) + df_test["s"]

In [122]:
df_val["s"] = df_val['c'].astype(str) + df_train["s"]

In [123]:
df_train["s"] = df_train["c"].astype(str) + df_train["s"]

In [124]:
df_train = df_train.drop('x', axis=1)
df_train = df_train.drop('b', axis=1)
df_train = df_train.drop('c', axis=1)

In [125]:
df_val = df_val.drop('x', axis=1)
df_val = df_val.drop('b', axis=1)
df_val = df_val.drop('c', axis=1)

In [126]:
df_test = df_test.drop('x', axis=1)
df_test = df_test.drop('b', axis=1)
df_test = df_test.drop('c', axis=1)

In [127]:
class divider(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(t,m,l) for t,m,l in zip(s["token"].values.tolist(),
                                                           s["m"].values.tolist(),
                                                           s["label"].values.tolist())]
        self.grouped = self.data.groupby("s").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [128]:
getter_trn = divider(df_train)
sentences_trn = [[s[0] for s in sent] for sent in getter_trn.sentences]

In [129]:
getter_val = divider(df_val)
sentences_val = [[s[0] for s in sent] for sent in getter_val.sentences]

In [130]:
getter_tst = divider(df_test)
sentences_tst = [[str(s[0]) for s in sent] for sent in getter_tst.sentences]

In [131]:
topic_trn = [[s[1] for s in sent] for sent in getter_trn.sentences]
labels_trn = [[s[2] for s in sent] for sent in getter_trn.sentences]

In [132]:
topic_val = [[s[1] for s in sent] for sent in getter_val.sentences]
labels_val = [[s[2] for s in sent] for sent in getter_val.sentences]

In [133]:
topic_tst = [[s[1] for s in sent] for sent in getter_tst.sentences]
labels_tst = [[s[2] for s in sent] for sent in getter_tst.sentences]

In [134]:
tags_vals = list(set(df_train["label"].values))

In [135]:
tags_vals = set(tags_vals)
tags_vals

{'O', 'TERM'}

In [136]:
tagtoid={'O': 0,
         'TERM' : 1,
         }

In [137]:
idtotag={tagtoid[key] : key for key in tagtoid.keys()}

In [138]:
device = torch.device("cuda" )if torch.cuda.is_available() else "cpu"
n_gpu = torch.cuda.device_count()

In [139]:
device

'cpu'

In [140]:
# Len of the sentence must be the same as the training model
# define maximum sentence length as 128
max_len  = 128
batch_num = 64
# load tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [141]:
sen_tst = []
for i in sentences_tst :
    for j in i:
        sen_tst.append(str(j))

In [142]:
sen_trn = []
for i in sentences_trn :
    for j in i:
        sen_trn.append(str(j))
lab_trn = []
for i in labels_trn :
    for j in i:
        lab_trn.append(str(j))   

In [143]:
sen_val = []
for i in sentences_val :
    for j in i:
        sen_val.append(str(j))
lab_val = []
for i in labels_val :
    for j in i:
        lab_val.append(str(j))        
        

In [144]:
tokenized_texts_tst = [sen_tst[i:i+40] for i in range(0, len(sen_tst), 40)]
tokenized_texts_tst ;   

In [145]:
tokenized_texts_trn = [sen_trn[i:i+40] for i in range(0, len(sen_trn), 40)]
tokenized_texts_trn ;   
word_piece_labels_trn = [lab_trn[i:i+40] for i in range(0, len(lab_trn), 40)]
word_piece_labels_trn ;  

In [146]:
tokenized_texts_val = [sen_val[i:i+40] for i in range(0, len(sen_val), 40)]
tokenized_texts_val ;   
word_piece_labels_val = [lab_val[i:i+40] for i in range(0, len(lab_val), 40)]
word_piece_labels_val ;   

In [147]:
# covert token to ids
input_ids_trn = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_trn],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_trn[0])

[  138   100 14940  4928  2196  1103   100  2860  1104  1956  1111  2510
  2233  1827   119  1966  1292  3584  1132  3417  1737   100  3776 14975
   117  1290  1152  1132  1304  7246  1106   100   120   100  1107  1103
  2013  1383   199   117]


In [148]:
# covert token to ids
input_ids_val = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_val],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_val[0])

[ 6177   199   100  1103  3000  9766  6440 11970  3442   119  6177   199
  2196  1103  1449   117  1103  2320 15305  2060  1180  1129  1277  2964
  1190  2166  8015   117  1105  2456  1122  1144   170  2964 20232  2060
   119  1130  1142  2526]


In [149]:
# covert token to ids
input_ids_tst = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_tst],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_tst[0])

[ 5893   117  1195 19774  1103   100  5625  6007   113   100 15982   114
   119   138   100  5625  6007  1110  1103  5625  1104   170   100   100
  1134  1110  1167  1190   170   100  2860   113   100   117  2908   100
  1104  1103  4177  1295]


In [150]:
input_ids_tst[15]

array([ 1106, 11552,  1107,   118, 19089,   100,   119,  1284,  6183,
        1115,  1185,  5531,  2745,   100,  3002,  1111,  1103,  1107,
         118, 19089,   100, 11432,   119,  1284,  2025,  1103, 23190,
        1104,  1103,  1295,  1104,  1551,  6211,  2607,   113,   100,
        1105,  4177,  6211,  1849])

In [151]:
tags_trn = pad_sequences([[tagtoid.get(l) for l in lab] for lab in word_piece_labels_trn],
                     maxlen=max_len, value=tagtoid["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_trn[0])

[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0]


In [152]:
tags_val = pad_sequences([[tagtoid.get(l) for l in lab] for lab in word_piece_labels_val],
                     maxlen=max_len, value=tagtoid["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_val[0])

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0]


In [153]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_trn = [[int(i>0) for i in ii] for ii in input_ids_trn]
attention_masks_trn[0];

In [154]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_val = [[int(i>0) for i in ii] for ii in input_ids_val]
attention_masks_val[0];

In [155]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_tst = [[int(i>0) for i in ii] for ii in input_ids_tst]
attention_masks_tst[0];

In [159]:
tr_inputs = torch.tensor(input_ids_trn)
val_inputs = torch.tensor(input_ids_val)
tst_inputs = torch.tensor(input_ids_tst)

tr_tags = torch.tensor(tags_trn)
val_tags = torch.tensor(tags_val)

tr_masks = torch.tensor(attention_masks_trn)
val_masks = torch.tensor(attention_masks_val)
tst_masks = torch.tensor(attention_masks_tst)

In [160]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [161]:
test_data = TensorDataset(tst_inputs, tst_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_num)

In [162]:
model = BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=len(tagtoid))

In [163]:
model = model.to(device)

In [164]:
# Set epoch and grad max num
epochs = 3
max_grad_norm = 1.0

In [165]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [166]:
# Fine tune model all layer parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
    ]

In [167]:
# TRAIN loop
model.train();

In [168]:
optimizer = AdamW(optimizer_grouped_parameters, lr=0.0001)

## Training phase

In [169]:
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  33%|███▎      | 1/3 [04:05<08:11, 245.89s/it]

Train loss: 0.13975565441484963


Epoch:  67%|██████▋   | 2/3 [08:12<04:06, 246.31s/it]

Train loss: 0.10328145108983985


Epoch: 100%|██████████| 3/3 [12:20<00:00, 246.81s/it]

Train loss: 0.0801732519342165


In [170]:
model_saving_address = 'models/final'
# Make dir if not exits
if not os.path.exists(model_saving_address):
    os.makedirs(model_saving_address)

In [171]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [172]:
# If we save using the predefined names, we can load using `from_pretrained`
model_file_location = os.path.join(model_saving_address, "pytorch_model.bin")
configuration_file_location = os.path.join(model_saving_address, "config.json")

# Save model into file
torch.save(model_to_save.state_dict(), model_file_location)
model_to_save.config.to_json_file(configuration_file_location)
tokenizer.save_vocabulary(model_saving_address)

('models/final/vocab.txt',)

In [173]:
model.eval();

## Evaluation phase

In [174]:
y_true = []
y_pred = []

for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label = batch
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,attention_mask=input_mask,)
        logits = outputs[0] 
   
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    label = label.to('cpu').numpy()
    
    input_mask = input_mask.to('cpu').numpy()
    
    for i,mask in enumerate(input_mask):
       # creating two empty lists.
        lst_1 = []
        lst_2 = []
        
        for j, m in enumerate(mask):
            if m:
                lst_1.append(idtotag[label[i][j]])
                lst_2.append(idtotag[logits[i][j]])
            else:
                break
        
            
        y_true.append(lst_1)
        y_pred.append(lst_2)
# the report
report = classification_report(y_true, y_pred,digits=4)


print("\n%s"%(report))
# F1 score
print("Final F1 score: %f"%(f1_score(y_true, y_pred)))
# Accuracy Score
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))



           precision    recall  f1-score   support

     TERM     0.7494    0.7257    0.7374      3656

micro avg     0.7494    0.7257    0.7374      3656
macro avg     0.7494    0.7257    0.7374      3656

Final F1 score: 0.737354
Accuracy score: 0.949546


## Training phase

In [175]:
# Model we trained before, the dir containing pytorch_model.bin and vocab.txt
save_model_address = 'models/final'

In [176]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tagtoid))

In [177]:
# Get model predict result
with torch.no_grad():
        outputs = save_model(tst_inputs, token_type_ids=None,attention_mask=None,)
        # For eval mode, the first result of outputs is logits

In [178]:
# Make logits into numpy type predict result
# The predict result contain each token's all tags predict result
predict_results = outputs[0].detach().cpu().numpy()

In [179]:
# shape of the predicted result
predict_results.shape

(1059, 40, 2)

In [180]:
from scipy.special import softmax

In [181]:
result_list = []
for pre_res in predict_results:
    result_arrays_soft =[]
    for i in pre_res:
        # Apply softmax on the result array.
        result_arrays_soft.append(softmax(i))
    result_list.append(np.argmax(result_arrays_soft,axis= -1))    

In [182]:
print(result_list[9])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [183]:
print(input_ids_tst[15])

[ 1106 11552  1107   118 19089   100   119  1284  6183  1115  1185  5531
  2745   100  3002  1111  1103  1107   118 19089   100 11432   119  1284
  2025  1103 23190  1104  1103  1295  1104  1551  6211  2607   113   100
  1105  4177  6211  1849]


In [184]:
print(tst_masks[9])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [186]:
answer = []
for j in range(len(result_list)): 
    for i, mark in enumerate(tst_masks[j]):
        if mark>0:
            if result_list[j][i] == 1 :   
                answer.append('TERM')      
            else :   
                answer.append('O')

In [187]:
len(answer)

42358

In [188]:
submission_df = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")

In [189]:
submission_df['label'] = pd.DataFrame(answer)

In [190]:
submission_df = submission_df.drop(['token'], axis=1)

In [191]:
# saving the output of testset in the csv
submission_df.to_csv("bert_base_cased.csv", index=False)